# Bernoulli Console With Random Orientation

Straight console, with starting point at $(0, 0, 0)$, length $L$, and random orientation of axis. The same concentrated forces and moments are applied at the free end in the coordinate system of the console. The results should be the same when understood in the frame of the console, independently of the random orientation angles.

In [5]:
L, d, t = 1000.0, 100.0, 5.0  #  geometry
Ex, nu = 210000.0, 0.25  #  material
penalty = 1e20  # penalty value for essential BCs

## Linear Solution

In [6]:
from linkeddeepdict.tools import getallfromkwargs
from neumann.linalg import Vector, linspace
from neumann import repeat
from polymesh.space import StandardFrame, PointCloud
from polymesh.utils.space import index_of_closest_point, index_of_furthest_point
from polymesh.utils.topology import L2_to_L3
from sigmaepsilon.fem.cells import B2, B3
from sigmaepsilon import Structure, LineMesh, PointData, BeamSection
import numpy as np
from latexdocs.utils import floatformatter

seed = 0  # integer or None
seed = None

rs = np.random.RandomState()
if isinstance(seed, int):
    rs.seed(seed)

formatter = floatformatter(sig=6)


def f2s(x):
    return formatter.format(x)


# section
section = BeamSection("CHS", d=d, t=t, n=32)
section.calculate_section_properties()
section_props = section.section_properties
A, Ix, Iy, Iz = getallfromkwargs(["A", "Ix", "Iy", "Iz"], **section_props)

# material
G = Ex / (2 * (1 + nu))
Hooke = np.array(
    [[Ex * A, 0, 0, 0], [0, G * Ix, 0, 0], [0, 0, Ex * Iy, 0], [0, 0, 0, Ex * Iz]]
)


def solve(n, angles, loads, celltype=B2):
    # space
    GlobalFrame = StandardFrame(dim=3)
    TargetFrame = GlobalFrame.rotate("Body", angles, "XYZ", inplace=False)

    # mesh
    p0 = np.array([0.0, 0.0, 0.0])
    p1 = np.array([L, 0.0, 0.0])
    coords = linspace(p0, p1, n + 1)
    points = PointCloud(coords, frame=TargetFrame)
    coords = points.show(GlobalFrame)
    topo = np.zeros((n, 2), dtype=int)
    topo[:, 0] = np.arange(n)
    topo[:, 1] = np.arange(n) + 1
    if celltype.NNODE == 3:
        coords, topo = L2_to_L3(coords, topo)
    i_first = index_of_closest_point(coords, np.array([0.0, 0.0, 0.0]))
    i_last = index_of_furthest_point(coords, np.array([0.0, 0.0, 0.0]))

    # essential boundary conditions
    fixity = np.zeros((coords.shape[0], 6)).astype(bool)
    fixity[i_first, :] = True
    fixity = fixity.astype(float) * penalty

    # natural boundary conditions
    loads = np.array(loads)
    nodal_loads = np.zeros((coords.shape[0], 6))
    vF = Vector(loads[:3], frame=TargetFrame).show(GlobalFrame)
    vM = Vector(loads[3:], frame=TargetFrame).show(GlobalFrame)
    nodal_loads[i_last, :3] = vF
    nodal_loads[i_last, 3:] = vM

    # pointdata
    pd = PointData(coords=coords, frame=GlobalFrame, loads=nodal_loads, fixity=fixity)

    # celldata
    frames = repeat(TargetFrame.axes, topo.shape[0])
    cd = celltype(topo=topo, material=Hooke, frames=frames)

    # set up mesh and structure
    mesh = LineMesh(pd, cd, frame=GlobalFrame)
    structure = Structure(mesh=mesh)

    structure.linear_static_analysis()

    dofsol = structure.nodal_dof_solution()
    u = np.zeros(6)
    u[:3] = Vector(dofsol[i_last, :3], frame=GlobalFrame).show(TargetFrame)
    u[3:] = Vector(dofsol[i_last, 3:], frame=GlobalFrame).show(TargetFrame)

    reactions = structure.reaction_forces()
    r = np.zeros(6)
    r[:3] = Vector(reactions[i_first, :3], frame=GlobalFrame).show(TargetFrame)
    r[3:] = Vector(reactions[i_first, 3:], frame=GlobalFrame).show(TargetFrame)

    forces = structure.internal_forces()
    f = np.zeros(6)
    f[:3] = forces[0, 0, :3, 0]
    f[3:] = forces[0, 0, 3:, 0]

    return u, r, f

## Tests

In [7]:
def absolute_relative_error(v_analytic, v_fem):
    return np.abs(100 * (v_analytic - v_fem) / v_analytic)

### Concentrated forces at the free end

In [8]:
Fx, Fy, Fz = 1.0, 1.0, 1.0
loads = [Fx, Fy, Fz, 0, 0, 0]
n = 2  # number of elements
cells = [B2, B3]

# analytical solutions
UX = Fx * L / (Ex * A)  # displacement at the free end
UY = Fy * L**3 / (3 * Ex * Iz)  # displacement at the free end
UZ = Fz * L**3 / (3 * Ex * Iy)  # displacement at the free end
UXX = 0.0  # rotation at the free end
UYY = -Fz * L**2 / (2 * Ex * Iy)  # rotation at the free end
UZZ = Fy * L**2 / (2 * Ex * Iz)  # rotation at the free end
RX = -Fx  # reaction force
RY = -Fy  # reaction force
RZ = -Fz  # reaction force
RXX = 0.0  # reaction moment
RYY = Fz * L  # reaction moment
RZZ = -Fy * L  # reaction moment
FX = Fx  # internal force at the support
FY = Fy  # internal force at the support
FZ = Fz  # internal force at the support
FXX = 0.0  # internal moment at the support
FYY = -Fz * L  # internal moment at the support
FZZ = Fy * L  # internal moment at the support

for cell in cells:
    for i in range(3):
        angles = np.random.rand(3) * np.pi * 2
        u, r, f = solve(n, angles, loads, cell)
        UX_fem, UXX_fem = u[0], u[3]
        RX_fem, RXX_fem = r[0], r[3]
        FX_fem, FXX_fem = f[0], f[3]
        UY_fem, UZZ_fem = u[1], u[5]
        RY_fem, RZZ_fem = r[1], r[5]
        FY_fem, FZZ_fem = f[1], f[5]
        UZ_fem, UYY_fem = u[2], u[4]
        RZ_fem, RYY_fem = r[2], r[4]
        FZ_fem, FYY_fem = f[2], f[4]
        print("---------------------- B{} ------------------\n".format(cell.NNODE))
        print("angles : {}".format(angles))
        print("UX  | Analytic : {}, FEM : {}".format(f2s(UX), f2s(UX_fem)))
        print("UY  | Analytic : {}, FEM : {}".format(f2s(UY), f2s(UY_fem)))
        print("UZ  | Analytic : {}, FEM : {}".format(f2s(UZ), f2s(UZ_fem)))
        print("UXX | Analytic : {}, FEM : {}".format(f2s(UXX), f2s(UXX_fem)))
        print("UYY | Analytic : {}, FEM : {}".format(f2s(UYY), f2s(UYY_fem)))
        print("UZZ | Analytic : {}, FEM : {}".format(f2s(UZZ), f2s(UZZ_fem)))
        print("RX  | Analytic : {}, FEM : {}".format(f2s(RX), f2s(RX_fem)))
        print("RY  | Analytic : {}, FEM : {}".format(f2s(RY), f2s(RY_fem)))
        print("RZ  | Analytic : {}, FEM : {}".format(f2s(RZ), f2s(RZ_fem)))
        print("RXX | Analytic : {}, FEM : {}".format(f2s(RXX), f2s(RXX_fem)))
        print("RYY | Analytic : {}, FEM : {}".format(f2s(RYY), f2s(RYY_fem)))
        print("RZZ | Analytic : {}, FEM : {}".format(f2s(RZZ), f2s(RZZ_fem)))
        print("FX  | Analytic : {}, FEM : {}".format(f2s(FX), f2s(FX_fem)))
        print("FY  | Analytic : {}, FEM : {}".format(f2s(FY), f2s(FY_fem)))
        print("FZ  | Analytic : {}, FEM : {}".format(f2s(FZ), f2s(FZ_fem)))
        print("FXX | Analytic : {}, FEM : {}".format(f2s(FXX), f2s(FXX_fem)))
        print("FYY | Analytic : {}, FEM : {}".format(f2s(FYY), f2s(FYY_fem)))
        print("FZZ | Analytic : {}, FEM : {}".format(f2s(FZZ), f2s(FZZ_fem)))
        print("\n")

---------------------- B2 ------------------

angles : [3.23206358 4.62435511 3.18613505]
UX  | Analytic : 3.21167e-06, FEM : 3.21167e-06
UY  | Analytic : 0.00095245, FEM : 0.00095245
UZ  | Analytic : 0.00095245, FEM : 0.00095245
UXX | Analytic : 0, FEM : -3.4089e-22
UYY | Analytic : -1.42867e-06, FEM : -1.42867e-06
UZZ | Analytic : 1.42867e-06, FEM : 1.42867e-06
RX  | Analytic : -1, FEM : -1
RY  | Analytic : -1, FEM : -1
RZ  | Analytic : -1, FEM : -1
RXX | Analytic : 0, FEM : 2.41797e-13
RYY | Analytic : 1000, FEM : 1000
RZZ | Analytic : -1000, FEM : -1000
FX  | Analytic : 1, FEM : 1
FY  | Analytic : 1, FEM : 1
FZ  | Analytic : 1, FEM : 1
FXX | Analytic : 0, FEM : -2.2233e-13
FYY | Analytic : -1000, FEM : -1000
FZZ | Analytic : 1000, FEM : 1000


---------------------- B2 ------------------

angles : [0.25596971 1.59319127 1.17011576]
UX  | Analytic : 3.21167e-06, FEM : 3.21167e-06
UY  | Analytic : 0.00095245, FEM : 0.00095245
UZ  | Analytic : 0.00095245, FEM : 0.00095245
UXX | Analyt

### Concentrated moments at the free end

In [9]:
Mx, My, Mz = 1.0, 1.0, 1.0
loads = [0, 0, 0, Mx, My, Mz]
n = 2  # number of elements
cells = [B2, B3]

# analytical solutions
UX = 0.0  # displacement at the free end
UY = Mz * L**2 / (2 * Ex * Iz)  # displacement at the free end
UZ = -My * L**2 / (2 * Ex * Iy)  # displacement at the free end
UXX = Mx * L / (G * Ix)  # rotation at the free end
UYY = My * L / (Ex * Iy)  # rotation at the free end
UZZ = Mz * L / (Ex * Iz)  # rotation at the free end
RX = 0.0  # reaction force
RY = 0.0  # reaction force
RZ = 0.0  # reaction force
RXX = -Mx  # reaction moment
RYY = -My  # reaction moment
RZZ = -Mz  # reaction moment
FX = 0.0  # internal force at the support
FY = 0.0  # internal force at the support
FZ = 0.0  # internal force at the support
FXX = Mx  # internal moment at the support
FYY = My  # internal moment at the support
FZZ = Mz  # internal moment at the support

for cell in cells:
    for i in range(3):
        angles = np.random.rand(3) * np.pi * 2
        u, r, f = solve(n, angles, loads, cell)
        UX_fem, UXX_fem = u[0], u[3]
        RX_fem, RXX_fem = r[0], r[3]
        FX_fem, FXX_fem = f[0], f[3]
        UY_fem, UZZ_fem = u[1], u[5]
        RY_fem, RZZ_fem = r[1], r[5]
        FY_fem, FZZ_fem = f[1], f[5]
        UZ_fem, UYY_fem = u[2], u[4]
        RZ_fem, RYY_fem = r[2], r[4]
        FZ_fem, FYY_fem = f[2], f[4]
        print("---------------------- B{} ------------------\n".format(cell.NNODE))
        print("angles : {}".format(angles))
        print("UX  | Analytic : {}, FEM : {}".format(f2s(UX), f2s(UX_fem)))
        print("UY  | Analytic : {}, FEM : {}".format(f2s(UY), f2s(UY_fem)))
        print("UZ  | Analytic : {}, FEM : {}".format(f2s(UZ), f2s(UZ_fem)))
        print("UXX | Analytic : {}, FEM : {}".format(f2s(UXX), f2s(UXX_fem)))
        print("UYY | Analytic : {}, FEM : {}".format(f2s(UYY), f2s(UYY_fem)))
        print("UZZ | Analytic : {}, FEM : {}".format(f2s(UZZ), f2s(UZZ_fem)))
        print("RX  | Analytic : {}, FEM : {}".format(f2s(RX), f2s(RX_fem)))
        print("RY  | Analytic : {}, FEM : {}".format(f2s(RY), f2s(RY_fem)))
        print("RZ  | Analytic : {}, FEM : {}".format(f2s(RZ), f2s(RZ_fem)))
        print("RXX | Analytic : {}, FEM : {}".format(f2s(RXX), f2s(RXX_fem)))
        print("RYY | Analytic : {}, FEM : {}".format(f2s(RYY), f2s(RYY_fem)))
        print("RZZ | Analytic : {}, FEM : {}".format(f2s(RZZ), f2s(RZZ_fem)))
        print("FX  | Analytic : {}, FEM : {}".format(f2s(FX), f2s(FX_fem)))
        print("FY  | Analytic : {}, FEM : {}".format(f2s(FY), f2s(FY_fem)))
        print("FZ  | Analytic : {}, FEM : {}".format(f2s(FZ), f2s(FZ_fem)))
        print("FXX | Analytic : {}, FEM : {}".format(f2s(FXX), f2s(FXX_fem)))
        print("FYY | Analytic : {}, FEM : {}".format(f2s(FYY), f2s(FYY_fem)))
        print("FZZ | Analytic : {}, FEM : {}".format(f2s(FZZ), f2s(FZZ_fem)))
        print("\n")

---------------------- B2 ------------------

angles : [0.88463265 5.23811967 0.86667723]
UX  | Analytic : 0, FEM : -5.41448e-23
UY  | Analytic : 1.42867e-06, FEM : 1.42867e-06
UZ  | Analytic : -1.42867e-06, FEM : -1.42867e-06
UXX | Analytic : 3.57169e-09, FEM : 3.57169e-09
UYY | Analytic : 2.85735e-09, FEM : 2.85735e-09
UZZ | Analytic : 2.85735e-09, FEM : 2.85735e-09
RX  | Analytic : 0, FEM : -3.64359e-18
RY  | Analytic : 0, FEM : 7.20195e-19
RZ  | Analytic : 0, FEM : 5.7085e-18
RXX | Analytic : -1, FEM : -1
RYY | Analytic : -1, FEM : -1
RZZ | Analytic : -1, FEM : -1
FX  | Analytic : 0, FEM : -5.68403e-32
FY  | Analytic : 0, FEM : -7.77005e-18
FZ  | Analytic : 0, FEM : -7.69051e-18
FXX | Analytic : 1, FEM : 1
FYY | Analytic : 1, FEM : 1
FZZ | Analytic : 1, FEM : 1


---------------------- B2 ------------------

angles : [0.39576442 1.43316997 0.72594995]
UX  | Analytic : 0, FEM : -1.4169e-22
UY  | Analytic : 1.42867e-06, FEM : 1.42867e-06
UZ  | Analytic : -1.42867e-06, FEM : -1.42867e